In [6]:
load("~/ma611-code/solid_angle.sage")

In [7]:
def check_sign_consistency(v):
    r"""
    input matrix v, whose rows are v[0], ..., v[n-1].
    Mn(v)_{i,j} =  -|vi \cdot vj| for i \neq j, and = vi\cdot vj if i=j.
    Check if Mn(v) be written as (v')^T * (v'),
    with v'[i] = + v'[i] or - v'[i].

    # Example 3.4 from 2010-Gourion-Seeger-Deterministic and stochastic methods
    sage: v = matrix([[1,-1,-1,1],[5,1,7,5],[-4,4,1,4],[-4,-5,8,4]])
    sage: check_sign_consistency(v)
    False
    """
    n = v.nrows()
    s = [0]*n
    for i in range(n):
        if s[i] == 0:
            s[i] = 1
        for j in range(i+1, n):
            if v[i] * v[j] < 0:
                if s[j] == -s[i]:
                    return False
                s[j] = s[i]
            elif v[i] * v[j] > 0:
                if s[j] == s[i]:
                    return False
                s[j] = -s[i]
    return True

In [ ]:
does check sign consistency correpond to positive definiteness? the gourion seeger example 3.4 gives that it is,
bu sign consistency says false? the is_m_alpha_posdef also gives true

In [6]:
A = matrix([[1,-1,0],[2,1,1],[-1,0,0]])
check_sign_consistency(A)

False

In [37]:
A = matrix([[1,-1,0],[2,1,1],[-1,0,0]])
is_M_alpha_posdef(A)  

INFO: Associated Matrix:
[                1.0 -0.2886751345948128 -0.7071067811865475]
[-0.2886751345948128                 1.0 -0.8164965809277259]
[-0.7071067811865475 -0.8164965809277259                 1.0]


False

In [38]:
#Example 3.4 Gourion
A=matrix([[1,-1,-1,1],[5,1,7,5],[-4,4,1,4],[-4,-5,8,4]]) 
check_sign_consistency(A)

False

In [39]:
A=matrix([[1,-1,-1,1],[5,1,7,5],[-4,4,1,4],[-4,-5,8,4]])
is_M_alpha_posdef(A)  

INFO: Associated Matrix:
[                 1.0                 -0.1 -0.35714285714285715 -0.13636363636363635]
[                -0.1                  1.0 -0.15714285714285714  -0.4636363636363636]
[-0.35714285714285715 -0.15714285714285714                  1.0  -0.2597402597402597]
[-0.13636363636363635  -0.4636363636363636  -0.2597402597402597                  1.0]


True

In [8]:
def generate_cones_decomposition(v, h=None, w=None, s=1, tridiag=True):
    r"""
    # Example 3.4 from 2010-Gourion-Seeger-Deterministic and stochastic methods
    sage: v = matrix([[1,-1,-1,1],[5,1,7,5],[-4,4,1,4],[-4,-5,8,4]])
    sage: list(generate_cones_decomposition(v, tridiag=False))
     [(
    [      1      -1      -1       1]   
    [      5       1       7       5]   
    [   17/2    13/2    37/2    33/2]   
    [-265/87 -740/87  370/87  -35/29], 1
    ),
     (
    [      1      -1      -1       1]    
    [      5       1       7       5]    
    [    7/2    -7/2    37/2    23/2]    
    [-265/67 -740/67  370/67 -105/67], -1
    ),
     (
    [    1    -1    -1     1]    
    [   -4     4     1     4]    
    [ 17/5  13/5  37/5  33/5]    
    [  8/5  37/5 -37/5  -8/5], -1
    ),
     (
    [     1     -1     -1      1]   
    [    -4     -5      8      4]   
    [   7/3   -7/3   37/3   23/3]   
    [465/79 720/79 370/79 625/79], 1
    ),
     (
    [      1      -1      -1       1]   
    [     -4      -5       8       4]   
    [    8/3    37/3   -37/3    -8/3]   
    [465/109 720/109 370/109 625/109], 1
    )]
    sage: list(generate_cones_decomposition(v, tridiag=True))
    [(
    [      1      -1      -1       1]   
    [      5       1       7       5]   
    [   17/2    13/2    37/2    33/2]   
    [-265/87 -740/87  370/87  -35/29], 1
    ),
     (
    [      1      -1      -1       1]    
    [      5       1       7       5]    
    [    7/2    -7/2    37/2    23/2]    
    [-265/67 -740/67  370/67 -105/67], -1
    ),
     (
    [        1        -1        -1         1]    
    [       -4         4         1         4]    
    [     17/5      13/5      37/5      33/5]    
    [      5/9  1010/153 -1480/153   -185/51], -1
    ),
     (
    [       1       -1       -1        1]   
    [      -4        4        1        4]   
    [     8/5     37/5    -37/5     -8/5]   
    [   85/47  1010/47 -1480/47  -555/47], 1
    ),
     (
    [     1     -1     -1      1]   
    [    -4     -5      8      4]   
    [   7/3   -7/3   37/3   23/3]   
    [465/79 720/79 370/79 625/79], 1
    ),
     (
    [      1      -1      -1       1]   
    [     -4      -5       8       4]   
    [    8/3    37/3   -37/3    -8/3]   
    [465/109 720/109 370/109 625/109], 1
    )]
    """
    #print v, h, w, s
    #if v.nrows() == 2: ? if check_sign_consistency(v)): ?
    if (v.nrows() <= 2) or ((not tridiag) and (check_sign_consistency(v))):
        if w is None:
            yield((v, s))
        else:
            yield((w.stack(v), s))
    else:
        n = v.nrows()
        if h is None:
            max_num_orth = -1
            for i in range(n):
                num_orth = [v[i]*v[j] for j in range(n)].count(0)
                if num_orth > max_num_orth:
                    max_num_orth = num_orth
                    #breakpoint()
                    h = i
        if w is None:
            ww = matrix(v[h])
        else:
            ww = w.stack(v[h])
        num_orth = [v[h]*v[j] for j in range(n)].count(0)
        if num_orth == n-1:
            u = v.delete_rows([h])
            for vs in generate_cones_decomposition(u, h=None, w=ww, s=s, tridiag=tridiag):
                yield vs
        else:
            for i in range(n):
                if (i == h) or (v[i]*v[h]==0):
                    continue
                u = matrix(v[i])
                if v[i]*v[h] > 0:
                    si = s
                    for j in range(i):
                        if (j != h) and (v[j]*v[h]>0):
                            si = -si
                    for k in range(n):
                        if (k == h) or (k == i):
                            continue
                        if (k < i) and (v[k]*v[h]>0):
                            eik = -1
                        else:
                            eik = 1
                        projvk = v[k]-(v[k]*v[h])/(v[i]*v[h]) * v[i]
                        u = u.stack(eik * projvk)
                    for vs in generate_cones_decomposition(u, h=0, w=ww, s=si, tridiag=tridiag):
                        yield vs
                elif v[i]*v[h] < 0:
                    si = s
                    for j in range(i+1, n):
                        if (j != h) and (v[j]*v[h]<0):
                            si = -si
                    for k in range(n):
                        if (k == h) or (k == i):
                            continue
                        if (k > i) and (v[k]*v[h]<0):
                            eik = -1
                        else:
                            eik = 1
                        projvk = v[k]-(v[k]*v[h])/(v[i]*v[h]) * v[i]
                        u = u.stack(eik * projvk)
                    for vs in generate_cones_decomposition(u, h=0, w=ww, s=si, tridiag=tridiag):
                        yield vs

In [12]:
A=matrix([[1,2,3],[1,0,1],[1,1,1]])
list(generate_cones_decomposition(A, tridiag=False))

> <ipython-input-8-35bb3877513c>(90)generate_cones_decomposition()
-> h = i
(Pdb) next
> <ipython-input-8-35bb3877513c>(85)generate_cones_decomposition()
-> for i in range(n):
(Pdb) print(h)
0
(Pdb) next
> <ipython-input-8-35bb3877513c>(86)generate_cones_decomposition()
-> num_orth = [v[i]*v[j] for j in range(n)].count(Integer(0))
(Pdb) next
> <ipython-input-8-35bb3877513c>(87)generate_cones_decomposition()
-> if num_orth > max_num_orth:
(Pdb) next
> <ipython-input-8-35bb3877513c>(85)generate_cones_decomposition()
-> for i in range(n):
(Pdb) next
> <ipython-input-8-35bb3877513c>(86)generate_cones_decomposition()
-> num_orth = [v[i]*v[j] for j in range(n)].count(Integer(0))
(Pdb) next
> <ipython-input-8-35bb3877513c>(87)generate_cones_decomposition()
-> if num_orth > max_num_orth:
(Pdb) next
> <ipython-input-8-35bb3877513c>(85)generate_cones_decomposition()
-> for i in range(n):
(Pdb) next
> <ipython-input-8-35bb3877513c>(91)generate_cones_decomposition()
-> if w is None:
(Pdb) print(h)

[(
[   1    2    3]   
[   1    0    1]   
[-1/2    1 -1/2], 1
),
 (
[   1    2    3]    
[   1    1    1]    
[-1/3  2/3 -1/3], -1
)]

Internal StopIteration: False
> /Users/xhaferrama/Desktop/sage/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py(3254)run_ast_nodes()
-> if (await self.run_code(code, result,  async_=asy)):
(Pdb) next
> /Users/xhaferrama/Desktop/sage/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py(3246)run_ast_nodes()
-> for node,mode in to_run:
(Pdb) quit


BdbQuit: 

Below showing how s=-1 works

In [2]:
def sza(v, h=None, w=None, s=1, tridiag=True):
    if (v.nrows() <= 2) or ((not tridiag) and (check_sign_consistency(v))):
        if w is None:
            yield((v, s))
        else:
            yield((w.stack(v), s))
    else:
        n = v.nrows()
        if h is None:
            max_num_orth = -1
            for i in range(n):
                num_orth = [v[i]*v[j] for j in range(n)].count(0)
                if num_orth > max_num_orth:
                    max_num_orth = num_orth
                    #breakpoint()
                    h = i
        if w is None:
            ww = matrix(v[h])
        else:
            ww = w.stack(v[h])
        num_orth = [v[h]*v[j] for j in range(n)].count(0)
        if num_orth == n-1:
            u = v.delete_rows([h])
            for vs in sza(u, h=None, w=ww, s=s, tridiag=tridiag):
                yield vs
        else:
            for i in range(n):
                if (i == h) or (v[i]*v[h]==0):
                    continue
                u = matrix(v[i])
                if v[i]*v[h] > 0:
                    si = s
                    for j in range(i):
                        if (j != h) and (v[j]*v[h]>0):
                            si = -si
                    for k in range(n):
                        if (k == h) or (k == i):
                            continue
                        if (k < i) and (v[k]*v[h]>0):
                            eik = -1
                        else:
                            eik = 1
                        projvk = v[k]-(v[k]*v[h])/(v[i]*v[h]) * v[i]
                        u = u.stack(eik * projvk)
                    for vs in sza(u, h=0, w=ww, s=si, tridiag=tridiag):
                        yield vs
                elif v[i]*v[h] < 0:
                    si = s
                    for j in range(i+1, n):
                        if (j != h) and (v[j]*v[h]<0):
                            si = -si
                    for k in range(n):
                        if (k == h) or (k == i):
                            continue
                        if (k > i) and (v[k]*v[h]<0):
                            eik = -1
                        else:
                            eik = 1
                        projvk = v[k]-(v[k]*v[h])/(v[i]*v[h]) * v[i]
                        u = u.stack(eik * projvk)
                    for vs in sza(u, h=0, w=ww, s=si, tridiag=tridiag):
                        yield vs

In [ ]:
A=matrix([[1,-1,0],[2,1,1],[-1,0,0]])
list(sza(A, s=-1))

> <ipython-input-2-71663dc9ac53>(16)sza()
-> h = i
(Pdb) next
> <ipython-input-2-71663dc9ac53>(11)sza()
-> for i in range(n):
(Pdb) next
> <ipython-input-2-71663dc9ac53>(12)sza()
-> num_orth = [v[i]*v[j] for j in range(n)].count(Integer(0))
(Pdb) next
> <ipython-input-2-71663dc9ac53>(13)sza()
-> if num_orth > max_num_orth:
(Pdb) next
> <ipython-input-2-71663dc9ac53>(11)sza()
-> for i in range(n):
(Pdb) next
> <ipython-input-2-71663dc9ac53>(12)sza()
-> num_orth = [v[i]*v[j] for j in range(n)].count(Integer(0))
(Pdb) next
> <ipython-input-2-71663dc9ac53>(13)sza()
-> if num_orth > max_num_orth:
(Pdb) next
> <ipython-input-2-71663dc9ac53>(11)sza()
-> for i in range(n):
(Pdb) next
> <ipython-input-2-71663dc9ac53>(17)sza()
-> if w is None:
(Pdb) next
> <ipython-input-2-71663dc9ac53>(18)sza()
-> ww = matrix(v[h])
(Pdb) next
> <ipython-input-2-71663dc9ac53>(21)sza()
-> num_orth = [v[h]*v[j] for j in range(n)].count(Integer(0))
(Pdb) next
> <ipython-input-2-71663dc9ac53>(22)sza()
-> if num_orth

In [8]:
A=matrix([[1,-1,0],[2,1,1],[-1,0,0]])
list(generate_cones_decomposition(A, s=-1, tridiag=False))

> <ipython-input-2-35bb3877513c>(90)generate_cones_decomposition()
-> h = i
(Pdb) next
> <ipython-input-2-35bb3877513c>(85)generate_cones_decomposition()
-> for i in range(n):
(Pdb) 
> <ipython-input-2-35bb3877513c>(86)generate_cones_decomposition()
-> num_orth = [v[i]*v[j] for j in range(n)].count(Integer(0))
(Pdb) 
> <ipython-input-2-35bb3877513c>(87)generate_cones_decomposition()
-> if num_orth > max_num_orth:
(Pdb) 
> <ipython-input-2-35bb3877513c>(85)generate_cones_decomposition()
-> for i in range(n):
(Pdb) 
> <ipython-input-2-35bb3877513c>(86)generate_cones_decomposition()
-> num_orth = [v[i]*v[j] for j in range(n)].count(Integer(0))
(Pdb) 
> <ipython-input-2-35bb3877513c>(87)generate_cones_decomposition()
-> if num_orth > max_num_orth:
(Pdb) 
> <ipython-input-2-35bb3877513c>(85)generate_cones_decomposition()
-> for i in range(n):
(Pdb) 
> <ipython-input-2-35bb3877513c>(91)generate_cones_decomposition()
-> if w is None:
(Pdb) 
> <ipython-input-2-35bb3877513c>(92)generate_cones_d

[(
[ 1 -1  0]    
[ 2  1  1]    
[ 1  1  1], -1
),
 (
[ 1 -1  0]    
[-1  0  0]    
[ 1  1  1], -1
)]

Internal StopIteration: False
> /Users/xhaferrama/Desktop/sage/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py(3254)run_ast_nodes()
-> if (await self.run_code(code, result,  async_=asy)):
(Pdb) quit


BdbQuit: 

In [3]:
A=matrix([[1,0],[0,1]])
list(generate_cones_decomposition(A, tridiag=False))

[(
[1 0]   
[0 1], 1
)]

Below: the two other vectors are on the same side of L so should have same sign? But in code, we get +1 for one and -1 for the other; but we get signed projections...

In [29]:
c_1 = Cone ([(1,0,1), (1,2,3), (1,1,1)])
c_2 = Cone([(-1,2,-1),(1,-2,1), (1,2,3), (-1,-2,-3)])
g = c_2.plot(wall_alpha=0.2, wall_color='#0000FF', ray_label=None, ray_thickness=0.01, wall_label=None) + c_1. plot(wall_alpha=0.2, wall_color='#00BFFF', ray_label=None, wall_label=None)
h=g + text3d("x", (4,0,0)) + text3d("y", (0,4,0)) + text3d("z", (0,0,2)) + line3d([(-2,0,0),(4,0,0)], color='black') + line3d([(0,-2,0),(0,4,0)], color='black') + line3d([(0,0,-4),(0,0,2)], color='black')
h + text3d("w_3", (1,2,3))+ text3d("w_2", (1,0,1))+ text3d("w_1", (1,1,1))


Graphics3d Object

In [33]:
#BV-decomp:
c_1 = Cone ([(-1,2,-1), (1,2,3), (1,0,1)])
c_2 = Cone([(1,2,3),(1,-2,1), (1,1,1)]) #blue
g = c_2.plot(wall_alpha=0.2, wall_color='#1956FC', ray_label=None, ray_color='red', ray_thickness=0.01, wall_label=None) + c_1. plot(wall_alpha=0.2, wall_color='#FFEF00', ray_label=None, wall_label=None)
h=g + text3d("x", (4,0,0)) + text3d("y", (0,4,0)) + text3d("z", (0,0,2)) + line3d([(-2,0,0),(4,0,0)], color='black') + line3d([(0,-2,0),(0,4,0)], color='black') + line3d([(0,0,-4),(0,0,2)], color='black')
h + text3d("w_3", (1,2,3))+ text3d("w_2", (1,0,1))+ text3d("w_1", (1,1,1))

Graphics3d Object

In [5]:
#code-decomp:
# c_1-c_2; yellow - blue
c_1 = Cone ([(1,2,3), (1,0,1), (-1,2,-1)])
c_2 = Cone([(1,2,3),(1,1,1), (-1,2,-1)]) #blue
g = c_2.plot(wall_alpha=0.2, wall_color='#1956FC', ray_label=None, ray_color='red', ray_thickness=0.01, wall_label=None) + c_1. plot(wall_alpha=0.2, wall_color='#FFEF00', ray_label=None, wall_label=None)
h=g + text3d("x", (4,0,0)) + text3d("y", (0,4,0)) + text3d("z", (0,0,2)) + line3d([(-2,0,0),(4,0,0)], color='black') + line3d([(0,-2,0),(0,4,0)], color='black') + line3d([(0,0,-4),(0,0,2)], color='black')
h

Graphics3d Object

Plotting BV decomp and Zhou decomp... for BV need to add mod LV terms, for Z, already accounted for

In [11]:
A=matrix([[1,-1,0],[2,1,1],[-1,0,0]])
list(generate_cones_decomposition(A, s=1))

[(
[ 1 -1  0]   
[ 2  1  1]   
[ 1  1  1], 1
),
 (
[ 1 -1  0]   
[-1  0  0]   
[ 1  1  1], 1
)]

In [16]:
c_A = Cone ([(1,-1,0), (2,1,1), (-1,0,0)])
g = c_A.plot(wall_alpha=0.2, wall_color='#0000FF', ray_label=None, ray_thickness=0.01, wall_label=None)
L = Cone ([(4,1,1), (-4,-1,-1), (1,-1,0),(-1,1,0)]) #yellow
h = L.plot(wall_alpha=0.2, wall_color='#EFFF06', ray_label=None, ray_thickness=0.01, wall_label=None)
g + h + text3d("x", (4,0,0)) + text3d("y", (0,4,0)) + text3d("z", (0,0,2)) + line3d([(-2,0,0),(4,0,0)], color='black') + line3d([(0,-2,0),(0,4,0)], color='black') + line3d([(0,0,-4),(0,0,2)], color='black')+ text3d("w_3", (1,-1,0))+ text3d("w_2", (2,1,1))+ text3d("w_1", (-1,0,0))


Graphics3d Object

In [13]:
#Zhou-Decomposition
c_1 = Cone ([(1,-1,0), (2,1,1), (1,1,1)])blue
c_2 = Cone([(1,-1,0),(-1,0,0), (1,1,1)]) purple
g = c_2.plot(wall_alpha=0.2, wall_color='#0000FF', ray_label=None, ray_thickness=0.01, wall_label=None) + c_1. plot(wall_alpha=0.2, wall_color='#00BFFF', ray_label=None, wall_label=None)
h=g + text3d("x", (4,0,0)) + text3d("y", (0,4,0)) + text3d("z", (0,0,2)) + line3d([(-2,0,0),(4,0,0)], color='black') + line3d([(0,-2,0),(0,4,0)], color='black') + line3d([(0,0,-4),(0,0,2)], color='black')
h + text3d("w_3", (1,-1,0))+ text3d("w_2", (2,1,1))+ text3d("w_1", (-1,0,0))


Graphics3d Object

In [18]:
#BV-decomp
c_1 = Cone ([(1,-1,0), (2,1,1), (-4,-1,-1)]) #blue
c_2 = Cone([(1,-1,0),(-1,0,0), (4,1,1)]) #purple
g = c_2.plot(wall_alpha=0.2, wall_color='#0000FF', ray_label=None, ray_thickness=0.01, wall_label=None) + c_1. plot(wall_alpha=0.2, wall_color='#00BFFF', ray_label=None, wall_label=None)
h=g + text3d("x", (4,0,0)) + text3d("y", (0,4,0)) + text3d("z", (0,0,2)) + line3d([(-2,0,0),(4,0,0)], color='black') + line3d([(0,-2,0),(0,4,0)], color='black') + line3d([(0,0,-4),(0,0,2)], color='black')
h + text3d("w_3", (1,-1,0))+ text3d("w_2", (2,1,1))+ text3d("w_1", (-1,0,0))
#mod L(V) term is visible


Graphics3d Object

Below is debugging for draft of solid_angle function that takes product

In [9]:
def generate_orthogonal_parts(A):
    n = A.nrows()
    k = 0
    u_indice_list = [0]
    w_indice_set = set(range(1, n))
    while k < len(u_indice_list):
        i = u_indice_list[k]
        for j in range(n):
            if (j in w_indice_set) and (A[i]*A[j] != 0):
                u_indice_list.append(j)
                w_indice_set.remove(j)
        k += 1
    u = A.delete_rows(w_indice_set)
    yield u
    if w_indice_set:
        w = A.matrix_from_rows(w_indice_set)
        for result in generate_orthogonal_parts(w):
            yield result

In [8]:
def solid_angle(A, posdef=None, eps=1e-6, deg=100):
    #breakpoint()
    if posdef is True:
        p=solid_angle_general(A, eps=eps, deg=deg, space="affine")
        return p
    else:
        solid_angles=[]
        orth = list(generate_orthogonal_parts(A))
        m = len(orth)
        for i in range(m):
            if is_M_alpha_posdef(orth[i]) is True:
                R=solid_angle_general(orth[i], eps=eps, deg=deg, space="affine")
                solid_angles.append(R)
            else:
                D = list(generate_cones_decomposition(orth[i]))
                d = len(D)
                sum_angles = 0
                for j in range(d):
                    if D[j][1]==1:
                        sum_angles += solid_angle_general(D[j][0], eps=eps,
                                                deg=deg, space="affine")
                    else:
                        sum_angles += -1*solid_angle_general(D[j][0], eps=eps,
                                                deg=deg, space="affine")
            solid_angles.append(sum_angles)
    return prod(solid_angles)

In [28]:
A=matrix([[1,-1,0],[2,1,1],[-1,0,0]])
solid_angle_3d(A)

0.3587200612139267

In [31]:
A=matrix([[1,-1,0],[2,1,1],[-1,0,0]])
solid_angle(A, deg=150)

INFO: Associated Matrix:
[                1.0 -0.2886751345948128 -0.7071067811865475]
[-0.2886751345948128                 1.0 -0.8164965809277259]
[-0.7071067811865475 -0.8164965809277259                 1.0]
INFO: Associated Matrix:
[                1.0 -0.2886751345948128                 0.0]
[-0.2886751345948128                 1.0 -0.9428090415820634]
[                0.0 -0.9428090415820634                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.012027413891614522]
INFO: Associated Matrix:
[                1.0 -0.7071067811865475                 0.0]
[-0.7071067811865475                 1.0 -0.5773502691896257]
[                0.0 -0.5773502691896257                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.34684523389214433]


0.3588726477837589

In [8]:
A=matrix([[1,-1,0],[2,1,1],[-1,0,0]])
solid_angle(A)

> <ipython-input-7-53e280c8395a>(4)solid_angle()
-> if posdef is True:
(Pdb) next
> <ipython-input-7-53e280c8395a>(8)solid_angle()
-> orth = list(generate_orthogonal_parts(A))
(Pdb) next
> <ipython-input-7-53e280c8395a>(9)solid_angle()
-> m = len(orth)
(Pdb) next
> <ipython-input-7-53e280c8395a>(10)solid_angle()
-> for i in range(m):
(Pdb) next
> <ipython-input-7-53e280c8395a>(11)solid_angle()
-> if is_M_alpha_posdef(orth[i]) is True:
(Pdb) next


INFO: Associated Matrix:
[                1.0 -0.2886751345948128 -0.7071067811865475]
[-0.2886751345948128                 1.0 -0.8164965809277259]
[-0.7071067811865475 -0.8164965809277259                 1.0]


> <ipython-input-7-53e280c8395a>(16)solid_angle()
-> D = list(generate_cones_decomposition(orth[i]))
(Pdb) next
> <ipython-input-7-53e280c8395a>(17)solid_angle()
-> d = len(D)
(Pdb) print(D)
[([ 1 -1  0]
[ 2  1  1]
[ 1  1  1], 1), ([ 1 -1  0]
[-1  0  0]
[ 1  1  1], 1)]
(Pdb) next
> <ipython-input-7-53e280c8395a>(18)solid_angle()
-> for j in range(d):
(Pdb) print(d)
2
(Pdb) next
> <ipython-input-7-53e280c8395a>(19)solid_angle()
-> sum_angles = Integer(0)
(Pdb) print(j)
0
(Pdb) next
> <ipython-input-7-53e280c8395a>(20)solid_angle()
-> if D[j][Integer(1)]==Integer(1):
(Pdb) print(sum_angles)
0
(Pdb) next
> <ipython-input-7-53e280c8395a>(21)solid_angle()
-> sum_angles += solid_angle_general(D[j][Integer(0)], eps=eps,
(Pdb) print(D[j][1])
1
(Pdb) next
> <ipython-input-7-53e280c8395a>(22)solid_angle()
-> deg=deg, space="affine")
(Pdb) print(sum_angles)
0
(Pdb) next
> <ipython-input-7-53e280c8395a>(21)solid_angle()
-> sum_angles += solid_angle_general(D[j][Integer(0)], eps=eps,
(Pdb) print(su

INFO: Associated Matrix:
[                1.0 -0.2886751345948128                 0.0]
[-0.2886751345948128                 1.0 -0.9428090415820634]
[                0.0 -0.9428090415820634                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.012266062796929114]


> <ipython-input-7-53e280c8395a>(23)solid_angle()
-> solid_angles.append(sum_angles)
(Pdb) print(sum_angles)
0.012266062796929114
(Pdb) next
> <ipython-input-7-53e280c8395a>(28)solid_angle()
-> logging.info("Solid angle sum for one cone: %s" % sum_angles)
(Pdb) next


...

INFO: Associated Matrix:
[                1.0 -0.7071067811865475                 0.0]
[-0.7071067811865475                 1.0 -0.5773502691896257]
[                0.0 -0.5773502691896257                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.34684523389214433]


> <ipython-input-7-53e280c8395a>(23)solid_angle()
-> solid_angles.append(sum_angles)
(Pdb) print(sum_angles)
0.34684523389214433
(Pdb) next
> <ipython-input-7-53e280c8395a>(28)solid_angle()
-> logging.info("Solid angle sum for one cone: %s" % sum_angles)
(Pdb) next


INFO: Solid angle sum for one cone: 0.34684523389214433


> <ipython-input-7-53e280c8395a>(18)solid_angle()
-> for j in range(d):
(Pdb) next
> <ipython-input-7-53e280c8395a>(10)solid_angle()
-> for i in range(m):
(Pdb) print(j)
1
(Pdb) next
> <ipython-input-7-53e280c8395a>(29)solid_angle()
-> return prod(solid_angles)
(Pdb) next
--Return--
> <ipython-input-7-53e280c8395a>(29)solid_angle()->0.004254425419736608
-> return prod(solid_angles)
(Pdb) next
--Call--
> /Users/xhaferrama/Desktop/sage/local/lib/python3.9/site-packages/IPython/core/displayhook.py(252)__call__()
-> def __call__(self, result=None):
(Pdb) print(prod)
*** NameError: name 'prod' is not defined
(Pdb) print(prod(solid_angles))
*** NameError: name 'prod' is not defined
(Pdb) quit


BdbQuit: 

In [5]:
A=matrix([[1,-1,0],[2,1,1],[-1,0,0]])
list(generate_cones_decomposition(A, tridiag=False))

[(
[ 1 -1  0]   
[ 2  1  1]   
[ 1  1  1], 1
),
 (
[ 1 -1  0]   
[-1  0  0]   
[ 1  1  1], 1
)]

In [10]:
A_1=matrix([[1,-1,0],[2,1,1],[1,1,1]])
solid_angle_3d(A_1)

0.011865046884419285

In [11]:
A_2=matrix([[1,-1,0],[-1,0,0],[1,1,1]])
solid_angle_3d(A_2)

0.34685501432950744

In [12]:
0.3587200612139267-0.34685501432950744

0.0118650468844193

In [ ]:
NEW STRATEGY: FIRST DO DECOMP
    - dont do solid_angle_general directly because dk if D[j][0] is posdef
    -check solid angle simplicial cone function: solid_angle_simplicial_cone(v, eps=1e-6, deg=100, normalized=True,tridiag=True):

In [8]:
 def sum_angles(A, eps=1e-6, deg=100):
    D = list(generate_cones_decomposition(A))
    d = len(D)
    sum_angles = 0
    for j in range(d):
        if D[j][1]==1:
            sum_angles += solid_angle_general(D[j][0], eps=eps,deg=deg, space="ambient")
        else:
            sum_angles += -1*solid_angle_general(D[j][0], eps=eps, deg=deg, space="ambient")
    return sum_angles

In [33]:
#A=matrix([[1,2,3],[1,0,1],[-1/2,1,-1/2]])
#solid_angle_3d(A) = 0.04075908094346588
#B=matrix([[1,2,3],[-1/3,2/3,-1/3],[1,1,1]])
#solid_angle_3d(B) = 0.030843964303606232
#angle for C should be A-B
#0.04075908094346588-0.030843964303606232=0.00991511663985965
C=matrix([[1,2,3],[1,0,1],[1,1,1]])
sum_angles(C)

INFO: Associated Matrix:
[                1.0 -0.7559289460184542                 0.0]
[-0.7559289460184542                 1.0 -0.5773502691896256]
[                0.0 -0.5773502691896256                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.04075941056052568]
INFO: Associated Matrix:
[                1.0 -0.9258200997725513                 0.0]
[-0.9258200997725513                 1.0                 0.0]
[                0.0                 0.0                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.030844774636990535]


0.009914635923535142

In [35]:
C=matrix([[1,2,3],[1,0,1],[1,1,1]])
solid_angle_3d(C)

0.009915116639859651

In [36]:
def solid_angle_measure(A, eps=1e-6, deg=100):
    if is_M_alpha_posdef(A) is True:
        return solid_angle_general(A, deg=deg, eps=eps)
    else:
        #breakpoint()
        solid_angle_factors=[]
        orth=list(generate_orthogonal_parts(A))
        m = len(orth)
        for i in range(m):
            t = sum_angles(orth[i], eps=eps, deg=deg)
            solid_angle_factors.append(t)
        return prod(solid_angle_factors)
            

In [16]:
A=matrix([[1,0,0],[0,1,0],[0,0,1]])
#should be 0.125
solid_angle_measure(A)

INFO: Associated Matrix:
[1.0 0.0 0.0]
[0.0 1.0 0.0]
[0.0 0.0 1.0]
INFO: Associated Matrix:
[1.0 0.0 0.0]
[0.0 1.0 0.0]
[0.0 0.0 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.12500000000000003]


0.12500000000000003

In [17]:
A = matrix([[2, sqrt(2), 3], [-1, 1, 2], [-3, 0, 5/4]])
#should be 0.01183076...
solid_angle_measure(A)

INFO: Associated Matrix:
[                1.0 -0.5707082198557699 -0.1787530775172654]
[-0.5707082198557699                 1.0 -0.6908817223234605]
[-0.1787530775172654 -0.6908817223234605                 1.0]
INFO: Associated Matrix:
[                1.0 -0.5707082198557699 -0.1787530775172654]
[-0.5707082198557699                 1.0 -0.6908817223234605]
[-0.1787530775172654 -0.6908817223234605                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.011831158032435412]


0.011831158032435412

In [18]:
A = matrix([[2, sqrt(2), 3], [-1, 1, 2], [-3, 0, 5/4]])
solid_angle_3d(A)

0.011830768935239901

In [19]:
A = matrix([[1/2, -1/2, -1/2, 1/2],[1/2, 1/10, 7/10, 1/2],
        ....:     [-4/7, 4/7, 1/7, 4/7], [-4/11, -5/11, 8/11, 4/11]])
solid_angle_measure(A)

INFO: Associated Matrix:
[                 1.0                 -0.1 -0.35714285714285715 -0.13636363636363635]
[                -0.1                  1.0 -0.15714285714285714  -0.4636363636363636]
[-0.35714285714285715 -0.15714285714285714                  1.0  -0.2597402597402597]
[-0.13636363636363635  -0.4636363636363636  -0.2597402597402597                  1.0]
INFO: Associated Matrix:
[                 1.0                 -0.1 -0.35714285714285715 -0.13636363636363635]
[                -0.1                  1.0 -0.15714285714285714  -0.4636363636363636]
[-0.35714285714285715 -0.15714285714285714                  1.0  -0.2597402597402597]
[-0.13636363636363635  -0.4636363636363636  -0.2597402597402597                  1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.04043924941007706]


0.04043924941007706

In [20]:
A = matrix([[1/2, -1/2, -1/2, 1/2],[1/2, 1/10, 7/10, 1/2],
        ....:     [-4/7, 4/7, 1/7, 4/7], [-4/11, -5/11, 8/11, 4/11]])
solid_angle_general(A)

INFO: Associated Matrix:
[                 1.0                 -0.1 -0.35714285714285715 -0.13636363636363635]
[                -0.1                  1.0 -0.15714285714285714  -0.4636363636363636]
[-0.35714285714285715 -0.15714285714285714                  1.0  -0.2597402597402597]
[-0.13636363636363635  -0.4636363636363636  -0.2597402597402597                  1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.04043924941007706]


0.04043924941007706

BELOW EXAMPLE PROBLEM

In [37]:
A = matrix([[1,-1,0],[2,1,1],[-1,0,0]])
#solid_angle_general(A) = 0.3582160872380887, associated matrix not posdef
#solid_angle_3d(A) 
# = 0.3587200612139267
solid_angle_measure(A, deg=250, eps=1e-11) # = default 0.35911129668907343; try adjusting epsilon,

INFO: Associated Matrix:
[                1.0 -0.2886751345948128 -0.7071067811865475]
[-0.2886751345948128                 1.0 -0.8164965809277259]
[-0.7071067811865475 -0.8164965809277259                 1.0]
INFO: Associated Matrix:
[                1.0 -0.2886751345948128                 0.0]
[-0.2886751345948128                 1.0 -0.9428090415820634]
[                0.0 -0.9428090415820634                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.01189584193669215]
INFO: Associated Matrix:
[                1.0 -0.7071067811865475                 0.0]
[-0.7071067811865475                 1.0 -0.5773502691896257]
[                0.0 -0.5773502691896257                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.3468550142299202]


0.35875085616661234

In [31]:
A = matrix([[1,0,0],[-1,0,0],[-1,3,1],[1,0,-1]])
#general gives 0.3012056062147818 and 3d gives 0.3012081...
solid_angle_measure(A)

INFO: Associated Matrix:
[                1.0                -1.0 -0.3015113445777637 -0.7071067811865475]
[               -1.0                 1.0 -0.3015113445777637 -0.7071067811865475]
[-0.3015113445777637 -0.3015113445777637                 1.0 -0.4264014327112209]
[-0.7071067811865475 -0.7071067811865475 -0.4264014327112209                 1.0]
INFO: Associated Matrix:
[               1.0                0.0                0.0]
[               0.0                1.0 -0.316227766016838]
[               0.0 -0.316227766016838                1.0]
INFO: Associated Matrix:
[               1.0                0.0                0.0]
[               0.0                1.0 -0.316227766016838]
[               0.0 -0.316227766016838                1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.09939568025802667, 0.09939568025802667]
INFO: Associated Matrix:
[                1.0 -0.3015113445777637                 0.0]
[-0.3015113445777637                 1.0                 0.0]
[  

0.5751405192812598

In [38]:
A = matrix([[1/2, -1/2, -1/2, 1/2],[1/2, 1/10, 7/10, 1/2], 
....:         ....:     [-4/7, 4/7, 1/7, 4/7], [-4/11, -5/11, 8/11, 4/11]]) 
solid_angle_measure(A, deg=0)

INFO: Associated Matrix:
[                 1.0                 -0.1 -0.35714285714285715 -0.13636363636363635]
[                -0.1                  1.0 -0.15714285714285714  -0.4636363636363636]
[-0.35714285714285715 -0.15714285714285714                  1.0  -0.2597402597402597]
[-0.13636363636363635  -0.4636363636363636  -0.2597402597402597                  1.0]
INFO: Associated Matrix:
[                 1.0                 -0.1 -0.35714285714285715 -0.13636363636363635]
[                -0.1                  1.0 -0.15714285714285714  -0.4636363636363636]
[-0.35714285714285715 -0.15714285714285714                  1.0  -0.2597402597402597]
[-0.13636363636363635  -0.4636363636363636  -0.2597402597402597                  1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.04870129870129872]


0.04870129870129872

In [10]:
def sum_angles(A, eps=1e-6, deg=100):
    breakpoint()
    D = list(generate_cones_decomposition(A))
    d = len(D)
    sum_angles = 0
    for j in range(d):
        if D[j][1] == 1:
            sum_angles += solid_angle_general(D[j][0], eps=eps, deg=deg,
                                              space="affine")
        else:
            sum_angles += -1*solid_angle_general(D[j][0], eps=eps, deg=deg,
                                                 space="affine")
    return sum_angles

In [40]:
A = matrix([[1,0,0],[-1,0,0],[-1,3,1],[1,0,-1]])   
sum_angles(A)

> <ipython-input-39-197a40550e88>(3)sum_angles()
-> D = list(generate_cones_decomposition(A))
(Pdb) next
> <ipython-input-39-197a40550e88>(4)sum_angles()
-> d = len(D)
(Pdb) print D
*** SyntaxError: Missing parentheses in call to 'print'. Did you mean print(D)?
(Pdb) print(D)
[([ 1  0  0]
[-1  0  0]
[ 0 -3 -1]
[ 0  0 -1], -1), ([    1     0     0]
[   -1     3     1]
[    0    -3    -1]
[    0  3/10 -9/10], 1), ([  1   0   0]
[ -1   3   1]
[  0   3   0]
[  0 1/3  -1], 1), ([ 1  0  0]
[ 1  0 -1]
[ 0  0 -1]
[ 0  3  0], 1)]
(Pdb) next
> <ipython-input-39-197a40550e88>(5)sum_angles()
-> sum_angles = Integer(0)
(Pdb) next
> <ipython-input-39-197a40550e88>(6)sum_angles()
-> for j in range(d):
(Pdb) next
> <ipython-input-39-197a40550e88>(7)sum_angles()
-> if D[j][Integer(1)] == Integer(1):
(Pdb) next
> <ipython-input-39-197a40550e88>(11)sum_angles()
-> sum_angles += -Integer(1)*solid_angle_general(D[j][Integer(0)], eps=eps, deg=deg,
(Pdb) next
> <ipython-input-39-197a40550e88>(12)sum_angles()

INFO: Associated Matrix:
[               1.0                0.0                0.0]
[               0.0                1.0 -0.316227766016838]
[               0.0 -0.316227766016838                1.0]
INFO: Associated Matrix:
[               1.0                0.0                0.0]
[               0.0                1.0 -0.316227766016838]
[               0.0 -0.316227766016838                1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.09939568025802667, 0.09939568025802667]


> <ipython-input-39-197a40550e88>(6)sum_angles()
-> for j in range(d):
(Pdb) print(sum_angles)
-0.19879136051605334
(Pdb) next
> <ipython-input-39-197a40550e88>(7)sum_angles()
-> if D[j][Integer(1)] == Integer(1):
(Pdb) print(j)
1
(Pdb) next
> <ipython-input-39-197a40550e88>(8)sum_angles()
-> sum_angles += solid_angle_general(D[j][Integer(0)], eps=eps, deg=deg,
(Pdb) next
> <ipython-input-39-197a40550e88>(9)sum_angles()
-> space="affine")
(Pdb) next
> <ipython-input-39-197a40550e88>(8)sum_angles()
-> sum_angles += solid_angle_general(D[j][Integer(0)], eps=eps, deg=deg,
(Pdb) print(sum_angles)
-0.19879136051605334
(Pdb) next


INFO: Associated Matrix:
[                1.0 -0.3015113445777637                 0.0]
[-0.3015113445777637                 1.0                 0.0]
[                0.0                 0.0                 1.0]
INFO: Associated Matrix:
[1.0 0.0 0.0]
[0.0 1.0 0.0]
[0.0 0.0 1.0]
INFO: Associated Matrix:
[                1.0 -0.9534625892455926                 0.0]
[-0.9534625892455926                 1.0                 0.0]
[                0.0                 0.0                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.14937252265055323, 0.12500000000000008, 0.2251476681318099]


> <ipython-input-39-197a40550e88>(6)sum_angles()
-> for j in range(d):
(Pdb) next
> <ipython-input-39-197a40550e88>(7)sum_angles()
-> if D[j][Integer(1)] == Integer(1):
(Pdb) print(sum_angles)
0.30072883026630987
(Pdb) next
> <ipython-input-39-197a40550e88>(8)sum_angles()
-> sum_angles += solid_angle_general(D[j][Integer(0)], eps=eps, deg=deg,
(Pdb) print(j)
2
(Pdb) next
> <ipython-input-39-197a40550e88>(9)sum_angles()
-> space="affine")
(Pdb) next
> <ipython-input-39-197a40550e88>(8)sum_angles()
-> sum_angles += solid_angle_general(D[j][Integer(0)], eps=eps, deg=deg,
(Pdb) next


INFO: Associated Matrix:
[                1.0 -0.3015113445777637                 0.0]
[-0.3015113445777637                 1.0 -0.9045340337332909]
[                0.0 -0.9045340337332909                 1.0]
INFO: Associated Matrix:
[               1.0                0.0                0.0]
[               0.0                1.0 -0.316227766016838]
[               0.0 -0.316227766016838                1.0]
INFO: Associated Matrix:
[                1.0 -0.9045340337332909                 0.0]
[-0.9045340337332909                 1.0  -0.316227766016838]
[                0.0  -0.316227766016838                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.029589198430402544, 0.09939568025802667, 0.020426149251754587]


> <ipython-input-39-197a40550e88>(6)sum_angles()
-> for j in range(d):
(Pdb) next
> <ipython-input-39-197a40550e88>(7)sum_angles()
-> if D[j][Integer(1)] == Integer(1):
(Pdb) print(sum_angles)
0.4501398582064937
(Pdb) print(j)
3
(Pdb) next
> <ipython-input-39-197a40550e88>(8)sum_angles()
-> sum_angles += solid_angle_general(D[j][Integer(0)], eps=eps, deg=deg,
(Pdb) next
> <ipython-input-39-197a40550e88>(9)sum_angles()
-> space="affine")
(Pdb) next
> <ipython-input-39-197a40550e88>(8)sum_angles()
-> sum_angles += solid_angle_general(D[j][Integer(0)], eps=eps, deg=deg,
(Pdb) next


INFO: Associated Matrix:
[                1.0 -0.7071067811865475                 0.0]
[-0.7071067811865475                 1.0                 0.0]
[                0.0                 0.0                 1.0]
INFO: Associated Matrix:
[                1.0 -0.7071067811865475                 0.0]
[-0.7071067811865475                 1.0                 0.0]
[                0.0                 0.0                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.06250033053738305, 0.06250033053738305]


> <ipython-input-39-197a40550e88>(6)sum_angles()
-> for j in range(d):
(Pdb) print(sum_angles)
0.5751405192812598
(Pdb) next
> <ipython-input-39-197a40550e88>(13)sum_angles()
-> return sum_angles
(Pdb) next
--Return--
> <ipython-input-39-197a40550e88>(13)sum_angles()->0.5751405192812598
-> return sum_angles
(Pdb) next
--Call--
> /Users/xhaferrama/Desktop/sage/local/lib/python3.9/site-packages/IPython/core/displayhook.py(252)__call__()
-> def __call__(self, result=None):
(Pdb) next
> /Users/xhaferrama/Desktop/sage/local/lib/python3.9/site-packages/IPython/core/displayhook.py(258)__call__()
-> self.check_for_underscore()
(Pdb) quit


BdbQuit: 

In [54]:
def tsum_angles(A, eps=1e-6, deg=100):
    A_list = simplicial_subcones_decomposition(A)
    b = len(A_list)
    for i in range(b):
        D_i = list(generate_cones_decomposition(A_list[i]))
        d = len(D_i)
        sum_angles = 0
        for j in range(d):
            if D_i[j][1] == 1:
                sum_angles += solid_angle_general(D_i[j][0], eps=eps, deg=deg, space="affine")
            else:
                sum_angles += -1*solid_angle_general(D_i[j][0], eps=eps, deg=deg,space="affine")
    return sum_angles

In [55]:
A = matrix([[1,0,0],[-1,0,0],[-1,3,1],[1,0,-1]])   
tsum_angles(A, deg=300, eps=1e-8)

INFO: Associated Matrix:
[                1.0 -0.3015113445777637                 0.0]
[-0.3015113445777637                 1.0 -0.9045340337332909]
[                0.0 -0.9045340337332909                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.029589635251848725]
INFO: Associated Matrix:
[                1.0 -0.7071067811865475                 0.0]
[-0.7071067811865475                 1.0                 0.0]
[                0.0                 0.0                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.062499996963738175]
INFO: Associated Matrix:
[                1.0 -0.3015113445777637                 0.0]
[-0.3015113445777637                 1.0 -0.9045340337332909]
[                0.0 -0.9045340337332909                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.021618555695646705]
INFO: Associated Matrix:
[                1.0 -0.7071067811865475                 0.0]
[-0.7071067811865475                 1.0                 0.0]
[    

0.2091185385403737

In [52]:
def solid_angle_simplicial_cone(v, eps=1e-6, deg=100, normalized=True,tridiag=True):
    # check condition for convergence
    # The following M is not normalized to have 1 on the diagonal
    M = v * v.transpose()
    d = v.nrows()
    for i in range(d):
        for j in range(d):
            if i != j:
                M[i,j] = - abs(M[i,j])
    if (not tridiag) and M.is_positive_definite():
        return product(solid_angle_general(orth_m, eps, deg, normalized,tridiag)
                       for orth_m in generate_orthogonal_parts(v))
        #return solid_angle_by_convergent_series(v, eps, deg, normalized,tridiag)
        #print "The matrix M is positive definite. Convergent series."
    # else:
    #     #print M
    #     print "Warning: the matrix M is not positive definite. Uncertain about the convergence of the series."
    answer = 1
    for orth_m in generate_orthogonal_parts(v):    
        angle = 0
        for (vi, si) in generate_cones_decomposition(orth_m, h=None, w=None, s=1, tridiag=tridiag):
            anglei = si
            for  orth_vi in generate_orthogonal_parts(vi):
                anglei *= solid_angle_general(orth_vi, eps, deg, normalized,tridiag)
            #anglei = solid_angle_by_convergent_series(vi, eps, deg, normalized,tridiag)
            angle += anglei
        answer = answer * angle
    return answer

In [63]:
A=matrix([[1,1,1],[-2,-1,0],[-2,0,-1]])
solid_angle_simplicial_cone(A)

INFO: Associated Matrix:
[                 1.0  -0.7745966692414835                  0.0]
[ -0.7745966692414835                  1.0 -0.31622776601683794]
[                 0.0 -0.31622776601683794                  1.0]
INFO: Associated Matrix:
[                 1.0  -0.7745966692414835                  0.0]
[ -0.7745966692414835                  1.0 -0.31622776601683794]
[                 0.0 -0.31622776601683794                  1.0]


0.15026119826694745

In [53]:
solid_angle_simplicial_cone(A)

INFO: Associated Matrix:
[ 1.0 -1.0]
[-1.0  1.0]
INFO: Associated Matrix:
[               1.0 -0.316227766016838]
[-0.316227766016838                1.0]
INFO: Associated Matrix:
[                1.0 -0.3015113445777637                 0.0]
[-0.3015113445777637                 1.0 -0.9534625892455926]
[                0.0 -0.9534625892455926                 1.0]
INFO: Associated Matrix:
[1.0]


RecursionError: maximum recursion depth exceeded in comparison

In [57]:
A = matrix([[1,0,0],[-1,0,0],[-1,3,1],[1,0,-1]])   
L=simplicial_subcones_decomposition(A)
L

[
[ 1  0  0]  [-1  0  0]
[-1  3  1]  [-1  3  1]
[ 1  0 -1], [ 1  0 -1]
]

In [58]:
solid_angle_3d(L[0])

0.09208963064104919

In [59]:
solid_angle_3d(L[1])

0.20911856053373415

In [60]:
solid_angle_measure(L[0])

INFO: Associated Matrix:
[                1.0 -0.3015113445777637 -0.7071067811865475]
[-0.3015113445777637                 1.0 -0.4264014327112209]
[-0.7071067811865475 -0.4264014327112209                 1.0]
INFO: Associated Matrix:
[                1.0 -0.3015113445777637 -0.7071067811865475]
[-0.3015113445777637                 1.0 -0.4264014327112209]
[-0.7071067811865475 -0.4264014327112209                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.09208935485799725]


0.09208935485799725

In [61]:
solid_angle_measure(L[1])

INFO: Associated Matrix:
[                1.0 -0.3015113445777637 -0.7071067811865475]
[-0.3015113445777637                 1.0 -0.4264014327112209]
[-0.7071067811865475 -0.4264014327112209                 1.0]
INFO: Associated Matrix:
[                1.0 -0.3015113445777637 -0.7071067811865475]
[-0.3015113445777637                 1.0 -0.4264014327112209]
[-0.7071067811865475 -0.4264014327112209                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.20911625135678452]


0.20911625135678452

In [62]:
solid_angle_measure(A)

INFO: Associated Matrix:
[                1.0                -1.0 -0.3015113445777637 -0.7071067811865475]
[               -1.0                 1.0 -0.3015113445777637 -0.7071067811865475]
[-0.3015113445777637 -0.3015113445777637                 1.0 -0.4264014327112209]
[-0.7071067811865475 -0.7071067811865475 -0.4264014327112209                 1.0]


NameError: name 'simplicial_subcones_decompositon' is not defined

In [77]:
def solid_angle_measure(A, eps=1e-6, deg=100, simplicial=False):
    if simplicial is True:
        if check_sign_consistency is True:
            return solid_angle_measure(A, eps=eps, deg=deg)
        else:
            solid_angle_factors=[]
            #breakpoint()
            orth=list(generate_orthogonal_parts(A))
            m = len(orth)
            for j in range(m):
                t = sum_angles(orth[j], eps=eps, deg=deg)
                solid_angle_factors.append(t)
            return prod(solid_angle_factors)
    else:
        A_list = simplicial_subcones_decomposition(A)
        b = len(A_list)
        p = 1
        for i in range(b):
            t_i = A_list[i]
            p *= solid_angle_measure(t_i, eps=eps, deg=deg, simplicial=True)
        return p

In [78]:
A = matrix([[1,-1,0],[2,1,1],[-1,0,0]])
solid_angle_measure(A)

INFO: Associated Matrix:
[                1.0 -0.2886751345948128                 0.0]
[-0.2886751345948128                 1.0 -0.9428090415820634]
[                0.0 -0.9428090415820634                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.012266062796929114]
INFO: Associated Matrix:
[                1.0 -0.7071067811865475                 0.0]
[-0.7071067811865475                 1.0 -0.5773502691896257]
[                0.0 -0.5773502691896257                 1.0]
INFO: Solid angle(s) of cones in Decomposition: [0.34684523389214433]


0.35911129668907343

In [79]:
solid_angle_3d(A)

0.3587200612139267

In [ ]:
def testr(A, eps=1e-6, deg=100, simplicial=False):  
    if not hasattr(A, 'nrows'):
        A = matrix(A)
    if simplicial is True:
        solid_angle_factors=[]
        if check_sign_consistency is True:
            solid_angle_factors.append(solid_angle_general(A))
        else:
            orth=list(generate_orthogonal_parts(A))
            m = len(orth)
            for j in range(m):
                t = sum_angles(orth[j], eps=eps, deg=deg)
                solid_angle_factors.append(t)
        return prod(solid_angle_factors)
    else:
        A_list = simplicial_subcones_decomposition(A)
        b = len(A_list)
        sum_sim = []
        for i in range(b):
            a_i = testr(A_list[i], eps=eps, deg=deg, simplicial=True)
            sum_sim.append(a_i)
        return sum(sum_sim)
    

In [ ]:
A = matrix([[1,0,0],[-1,0,0],[-1,3,1],[1,0,-1]])
testr(A)

FOR QUAL PRES: CONE ASSOC MATRIX IS NOT POSDEF

In [10]:
A=matrix([[1,0,0],[1,1,1],[1,1,0]])
A*A.transpose()

[1 1 1]
[1 3 2]
[1 2 2]

In [25]:
#BV decomp wrt line
c_1 = Cone ([(1,0,0), (1,1,1), (1,1,0)]) #blue
g = c_1. plot(wall_alpha=0.2, wall_color='#00BFFF', point_size=0, ray_label=None, wall_label=None)
h= g + text3d("x", (4,0,0)) + text3d("y", (0,4,0)) + text3d("z", (0,0,2)) + line3d([(-2,0,0),(4,0,0)], color='black') + line3d([(0,-2,0),(0,4,0)], color='black') + line3d([(0,0,-4),(0,0,2)], color='black')
j = h + text3d("w_1", (1,0,0)) + text3d("w_2", (1,1,1)) + text3d("w_3", (1,1,0))
j
j.save('plot_1.png')
